# [The Hard Hexagon model](@id demo_hardhexagon)

![logo](assets/hexagon.svg)

Tensor networks are a natural way to do statistical mechanics on a lattice.
As an example of this we will extract the central charge of the hard hexagon model.
This model is known to have central charge 0.8, and has very peculiar non-local (anyonic) symmetries.
Because TensorKit supports anyonic symmetries, so does MPSKit.
To follow the tutorial you need the following packages.

In [ ]:
using MPSKit, TensorKit, Plots, Polynomials
import TensorOperations; TensorOperations.disable_cache(); # hide

The [hard hexagon model](https://en.wikipedia.org/wiki/Hard_hexagon_model) is a 2-dimensional lattice model of a gas, where particles are allowed to be on the vertices of a triangular lattice, but no two particles may be adjacent.
This can be encoded in a transfer matrix with a local MPO tensor using anyonic symmetries as follows:

In [ ]:
P = Vect[FibonacciAnyon](:τ => 1)
O = TensorMap(ones, ComplexF64, P ⊗ P ← P ⊗ P)
blocks(O)[FibonacciAnyon(:I)] *= 0
mpo = DenseMPO(O);

## The leading boundary

One way to study statistical mechanics in infinite systems with tensor networks is by approximating the dominant eigenvector of the transfer matrix by an MPS. 
This dominant eigenvector contains a lot of hidden information.
For example, the free energy can be extracted by computing the expectation value of the mpo.
Additionally, we can compute the entanglement entropy as well as the correlation length of the state:

In [ ]:
D = 10
V = Vect[FibonacciAnyon](:τ => D, :I => D) # virtual space ≡ bond dimension
ψ₀ = InfiniteMPS([P], [V])
ψ, _ = leading_boundary(ψ₀, mpo, VUMPS(; verbose=false));

In [ ]:
F = real(first(expectation_value(ψ, mpo)))
S = real(first(entropy(ψ)))
ξ = correlation_length(ψ)
println("F = $F\tS = $S\tξ = $ξ")

## The scaling hypothesis

The dominant eigenvector is of course only an approximation. The finite bond dimension enforces a finite correlation length, which effectively introduces a length scale in the system. This can be exploited to formulate a [scaling hypothesis](https://arxiv.org/pdf/0812.2903.pdf), which in turn allows to extract the central charge.

First we need to know the entropy and correlation length at a bunch of different bond dimensions. Our approach will be to re-use the previous approximated dominant eigenvector, and then expanding its bond dimension and re-running VUMPS.
According to the scaling hypothesis we should have ``S \propto \frac{c}{6} log(ξ)``. Therefore we should find ``c`` using

In [ ]:
Ss = [S]
ξs = [ξ]
envs = environments(ψ, mpo)
for Ds in 5:5:30
    ψ, envs = changebonds(ψ, mpo, OptimalExpand(; trscheme=truncdim(5)), envs)
    ψ, envs, = leading_boundary(ψ, mpo, VUMPS(; verbose=false))
    push!(Ss, real(entropy(ψ)[1]))
    push!(ξs, correlation_length(ψ))
end

In [ ]:
f = fit(log.(ξs), 6 * Ss, 1)
c = f.coeffs[2]

In [ ]:
p = plot(; xlabel="logarithmic correlation length", ylabel="entanglement entropy")
p = plot(log.(ξs), Ss; seriestype=:scatter, label=nothing)
plot!(p, ξ -> f(ξ) / 6; label="fit")